In [19]:
import sys
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
df = pd.read_csv("dataset/train.csv")
X_toxic = df[(df['toxic'] == 1)]['comment_text']

text = ''.join(X_toxic)
text = text.lower()

In [4]:
chars = sorted(list(set(text)))
char_to_idx = dict((c, i) for i, c in enumerate(chars))
idx_to_char = dict((i, c) for i, c in enumerate(chars))

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x93', '\x94', '\xa0', '¢', '£', '¤', '¦', '§', '¨', '©', '\xad', '®', '¯', '°', '±', '²', '´', '·', '¸', '½', '¿', 'ß', 'à', 'á', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'í', 'ï', 'ñ', 'ó', 'ö', 'ù', 'ú', 'ü', 'þ', 'ą', 'ć', 'đ', 'ė', 'ě', 'ģ', 'ĥ', 'ħ', 'ı', 'ń', 'ņ', 'ō', 'œ', 'ś', 'ş', 'š', 'ţ', 'ũ', 'ŵ', 'ŷ', 'ż', 'ƒ', 'ǔ', 'ȳ', '̇', 'ά', 'ί', 'α', 'γ', 'δ', 'ε', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'φ', 'ω', 'ό', 'ύ', 'ώ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ц', 'ч', 'щ', 'ъ', 'ы', 'ь', 'я', 'љ', 'ּ', 'א', 'ב', 'ו', 'י', 'כ', 'ל', 'מ', 'ا', 

In [5]:
# summarize the loaded data
n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  4515494
Total Vocab:  347


In [15]:
seq_length = 100
X = []
y = []

for i in range(0, n_chars - seq_length, 1):
    sequence_in = text[i:i+ seq_length]
    sequence_out = text[i + seq_length]
    X.append([char_to_idx[char] for char in sequence_in])
    y.append(char_to_idx[sequence_out])
    
n_patterns = len(X)
print("Total Patterns: ", n_patterns)

Total Patterns:  4515394


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(y)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# fit the model
model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
4515394/4515394 [==============================] - 13165s 3ms/step - loss: 2.4030

Epoch 00001: loss improved from inf to 2.40299, saving model to weights-improvement-01-2.4030-bigger.hdf5
Epoch 2/10
4515394/4515394 [==============================] - 13218s 3ms/step - loss: 2.3467

Epoch 00002: loss improved from 2.40299 to 2.34672, saving model to weights-improvement-02-2.3467-bigger.hdf5
Epoch 3/10
4515394/4515394 [==============================] - 13109s 3ms/step - loss: 2.5975

Epoch 00003: loss did not improve from 2.34672
Epoch 4/10
 101184/4515394 [..............................] - ETA: 3:34:50 - loss: 2.7606

In [9]:
# load the network weights
filename = "weights-improvement-02-2.3467-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [47]:
# pick a random seed
start = numpy.random.randint(0, len(X)-1)
pattern = X[start]
print("Seed:")
print(''.join(idx_to_char[value] for value in pattern))

Seed:
pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig 


In [48]:
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = idx_to_char[index]
    seq_in = [idx_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig pig 